In [66]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt

In [34]:
df = pd.read_csv("./data/ten_years_of_earthquakes_TW.csv")
df.head(10)

,number,date,longitude_e,latitude_n,magnitude,depth,location,loc_full,loc_name
0,085,2023-12-31 05:57:10,121.839,24.6018,4.6,49.3,24.60N 121.84E,i.e. 16.3 km SSE of Yilan County(24.60N 121.84E,i.e. 16.3 km SSE of Yilan County)
1,Small area,2023-12-26 08:21:57,121.491,23.0357,3.7,26.6,23.04N 121.49E,i.e. 46.7 km NE of Taitung County(23.04N 121.49E,i.e. 46.7 km NE of Taitung County)
2,Small area,2023-12-26 06:19:24,121.755,24.3803,3.0,6.3,24.38N 121.76E,i.e. 38.8 km S of Yilan County(24.38N 121.76E,i.e. 38.8 km S of Yilan County)
3,Small area,2023-12-25 19:04:24,121.184,23.1407,3.4,14.5,23.14N 121.18E,i.e. 42.8 km N of Taitung County(23.14N 121.18E,i.e. 42.8 km N of Taitung County)
4,Small area,2023-12-25 11:49:35,121.919,24.4283,3.8,29.9,24.43N 121.92E,i.e. 37.1 km SSE of Yilan County(24.43N 121.92E,i.e. 37.1 km SSE of Yilan County)
5,Small area,2023-12-24 17:19:17,121.323,22.6363,4.4,21.0,22.64N 121.32E,i.e. 22.0 km SE of Taitung County(22.64N 121.32E,i.e. 22.0 km SE of Taitung County)
6,Small area,2023-12-24 05:21:18,121.308,22.6452,4.6,19.2,22.65N 121.31E,i.e. 20.2 km SE of Taitung County(22.65N 121.31E,i.e. 20.2 km SE of Taitung County)
7,Small area,2023-12-24 05:07:51,121.303,22.6458,4.5,19.0,22.65N 121.30E,i.e. 19.8 km SE of Taitung County(22.65N 121.30E,i.e. 19.8 km SE of Taitung County)
8,Small area,2023-12-23 10:32:34,122.141,24.2717,4.7,55.1,24.27N 122.14E,i.e. 61.4 km ENE of Hualien County(24.27N 122...,i.e. 61.4 km ENE of Hualien County)
9,Small area,2023-12-22 18:09:31,120.923,23.7777,3.6,28.1,23.78N 120.92E,i.e. 27.5 km ESE of Nantou County(23.78N 120.92E,i.e. 27.5 km ESE of Nantou County)


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6543 entries, 0 to 6542
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   number       6543 non-null   object 
 1   date         6543 non-null   object 
 2   longitude_e  6543 non-null   float64
 3   latitude_n   6543 non-null   float64
 4   magnitude    6543 non-null   float64
 5   depth        6543 non-null   float64
 6   location     6543 non-null   object 
 7   loc_full     6543 non-null   object 
 8   loc_name     6543 non-null   object 
dtypes: float64(4), object(5)
memory usage: 460.2+ KB


In [38]:
df.date = pd.to_datetime(df.date)

In [53]:
df['year'] = df.date.dt.year
df['month'] = df.date.dt.month_name
df['day'] = df.date.dt.day_name
df['hour'] = df.date.dt.hour


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6543 entries, 0 to 6542
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   number       6543 non-null   object        
 1   date         6543 non-null   datetime64[ns]
 2   longitude_e  6543 non-null   float64       
 3   latitude_n   6543 non-null   float64       
 4   magnitude    6543 non-null   float64       
 5   depth        6543 non-null   float64       
 6   location     6543 non-null   object        
 7   loc_full     6543 non-null   object        
 8   loc_name     6543 non-null   object        
 9   year         6543 non-null   int32         
 10  month        6543 non-null   object        
 11  day          6543 non-null   object        
 12  hour         6543 non-null   int32         
dtypes: datetime64[ns](1), float64(4), int32(2), object(6)
memory usage: 613.5+ KB


In [55]:
df.describe()

,date,longitude_e,latitude_n,magnitude,depth,year,hour
count,6543,6543.000000,6543.000000,6543.000000,6543.000000,6543.000000,6543.000000
mean,2018-04-18 20:37:33.409903872,121.412451,23.768137,3.706144,19.335672,2017.829742,11.431148
min,2013-01-01 03:00:53,118.520000,19.299500,1.300000,0.600000,2013.000000,0.000000
25%,2015-09-16 04:11:44,121.146500,23.260000,3.200000,8.500000,2015.000000,5.000000
50%,2018-02-07 09:42:35,121.503000,23.920000,3.600000,13.500000,2018.000000,11.000000
75%,2021-05-14 08:20:48,121.730000,24.260000,4.100000,21.600000,2021.000000,18.000000
max,2023-12-31 05:57:10,125.583000,25.986200,6.900000,268.600000,2023.000000,23.000000
std,NaN,0.523019,0.677398,0.745917,20.526199,3.182667,7.063549


In [82]:
lat_drift = df.groupby('year')['latitude_n'].agg('mean')
lat_drift

year
2013    23.920696
2014    23.846672
2015    23.961384
2016    23.799926
2017    23.665776
2018    23.857279
2019    23.827038
2020    23.656791
2021    23.728065
2022    23.503224
2023    23.625670
Name: latitude_n, dtype: float64

dtype('float64')

In [73]:
sns.scatterplot(x=lat_drift.index, y=lat_drift.values)

TypeError: Data source must be a DataFrame or Mapping, not <class 'pandas.core.series.Series'>.